In [3]:
from transformers import AutoTokenizer
import pandas as pd

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
df = pd.read_csv('../data/emrqa_all_data.csv')
df.head()

,question,context,answer,answer_start,evidence
0,What is the dosage of valproic acid,RECORD #91996\n150823816 | DMC | 60956989 | | ...,250 mg,7,throughout this pregnancy. She has been taking...
1,What is the patient's current dose does the pa...,RECORD #91996\n150823816 | DMC | 60956989 | | ...,250 mg,7,throughout this pregnancy. She has been taking...
2,What is the current dose of the patient's valp...,RECORD #91996\n150823816 | DMC | 60956989 | | ...,250 mg,7,throughout this pregnancy. She has been taking...
3,What is the current dose of valproic acid,RECORD #91996\n150823816 | DMC | 60956989 | | ...,250 mg,7,throughout this pregnancy. She has been taking...
4,How much valproic acid does the patient take p...,RECORD #91996\n150823816 | DMC | 60956989 | | ...,250 mg,7,throughout this pregnancy. She has been taking...


In [5]:
test = df.sample(5).reset_index(drop=True)
test

,question,context,answer,answer_start,evidence
0,Why has the patient been prescribed lasix,RECORD #573300\n219990133 | MHAMC | 03057354 |...,pulmonary edema,9,complication. Currently on 5 liters of O2 and ...
1,When was this patient's first documented diagn...,"Record date: 2077-08-04EDVISIT^42817774^AARON,...",2077-08-04,16,Record date: 2077-08-04\n\nEDVISIT^42817774^AA...
2,Does this woman have a history of hypertension,"Record date: 2076-09-25 OLIVIA PATTY HOPKINS,...",BP 142/60,904,BP 142/60 P 80 wt 174 lbs
3,MEDS RECORD,RECORD #808959\n\n\n022039090 | UGH | 34853160...,aspirin,1,3. Aspirin 325 mg p.o. daily.\n
4,Has this patient/employee ever been exposed to...,"Record date: 2079-01-14PICH 989 James Street, ...",Hyperlipidemia,12552,5) Pain management/ Narcotics log: Because of ...


In [6]:
row = test.iloc[0]
row

question                Why has the patient been prescribed lasix
context         RECORD #573300\n219990133 | MHAMC | 03057354 |...
answer                                            pulmonary edema
answer_start                                                    9
evidence        complication. Currently on 5 liters of O2 and ...
Name: 0, dtype: object

In [7]:
inputs = tokenizer(test['question'].tolist(), test['context'].tolist(), truncation='only_second',
                   stride=50, return_overflowing_tokens=True, return_offsets_mapping=True)
print(f"The 5 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 5 examples gave 47 features.
Here is where each comes from: [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4].


In [8]:
test['answer']

0    pulmonary edema
1         2077-08-04
2          BP 142/60
3            aspirin
4     Hyperlipidemia
Name: answer, dtype: object

In [9]:
next(i for i in range(len(inputs['input_ids'][0])) if inputs['input_ids'][0][i] == 102)

9

In [10]:
inputs['input_ids'][1]

[101,
 2339,
 2038,
 1996,
 5776,
 2042,
 16250,
 5869,
 7646,
 102,
 1045,
 1012,
 1040,
 1012,
 3558,
 7749,
 1024,
 4578,
 1998,
 3635,
 1019,
 2519,
 2184,
 5282,
 1010,
 10715,
 11382,
 10483,
 1012,
 8995,
 5751,
 1024,
 4860,
 5986,
 1012,
 1018,
 1010,
 2540,
 3446,
 4293,
 1010,
 17531,
 2157,
 2849,
 13029,
 1013,
 5786,
 1010,
 2187,
 2849,
 7287,
 1013,
 4293,
 1010,
 7722,
 2938,
 18924,
 5989,
 1003,
 2006,
 2282,
 2250,
 1012,
 18235,
 3372,
 1024,
 2566,
 12190,
 2050,
 1013,
 21418,
 22753,
 2302,
 3350,
 1997,
 8985,
 1013,
 2053,
 2482,
 4140,
 3593,
 7987,
 14663,
 2015,
 1013,
 10267,
 8525,
 15534,
 1012,
 3108,
 1024,
 2053,
 4297,
 19969,
 2015,
 1012,
 22935,
 1024,
 3180,
 6348,
 1998,
 25353,
 16033,
 10415,
 20227,
 1012,
 2035,
 29333,
 23894,
 10109,
 2007,
 1996,
 6453,
 1997,
 1996,
 2157,
 12759,
 2483,
 21877,
 10521,
 1010,
 2029,
 2001,
 2556,
 2011,
 2079,
 9397,
 3917,
 2069,
 1998,
 15219,
 14841,
 21102,
 23894,
 2020,
 2556,
 2011,
 2079,
 9397,

In [11]:
inputs.sequence_ids(0)

[None,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 None,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1

In [12]:
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = test['answer'][sample_idx]
    start_char = test["answer_start"][sample_idx]
    end_char = test["answer_start"][sample_idx] + len(answer)
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context]
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        adder = next(i for i in range(len(inputs['input_ids'][0])) if inputs['input_ids'][0][i] == 102)
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(adder + idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(adder + idx + 1)

start_positions, end_positions

([21,
  0,
  0,
  0,
  0,
  0,
  29,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  287,
  0,
  0,
  0,
  0,
  0,
  0,
  14,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  387,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [28,
  0,
  0,
  0,
  0,
  0,
  35,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  289,
  0,
  0,
  0,
  0,
  0,
  0,
  15,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  389,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0])

In [13]:
idx = 2
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = test['answer'][idx]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: BP 142/60, labels give: [CLS]


In [14]:
test['evidence'][2]

'BP 142/60 P 80 wt 174 lbs'

In [15]:
inputs["input_ids"][0]

[101,
 2339,
 2038,
 1996,
 5776,
 2042,
 16250,
 5869,
 7646,
 102,
 2501,
 1001,
 5401,
 22394,
 8889,
 20636,
 2683,
 21057,
 17134,
 2509,
 1064,
 1049,
 3511,
 2278,
 1064,
 6021,
 2692,
 28311,
 19481,
 2549,
 1064,
 1064,
 5345,
 28154,
 19841,
 2620,
 1064,
 1017,
 1013,
 2484,
 1013,
 2294,
 2260,
 1024,
 4002,
 1024,
 4002,
 2572,
 1064,
 20118,
 28228,
 2278,
 26261,
 27109,
 1064,
 2772,
 1064,
 4487,
 2015,
 1064,
 9634,
 3058,
 1024,
 1022,
 1013,
 2459,
 1013,
 2294,
 3189,
 3570,
 1024,
 2772,
 11889,
 3058,
 1024,
 1016,
 1013,
 2603,
 1013,
 2294,
 7052,
 1024,
 5440,
 1010,
 5099,
 10893,
 3406,
 9108,
 2326,
 1024,
 15050,
 5970,
 2326,
 1012,
 22137,
 1024,
 2000,
 11252,
 1012,
 4054,
 11889,
 11616,
 1024,
 3570,
 2695,
 20704,
 2099,
 2007,
 1037,
 2423,
 8292,
 20201,
 10764,
 1998,
 9298,
 2290,
 1060,
 2475,
 1013,
 12967,
 1012,
 2060,
 22939,
 26745,
 8583,
 1024,
 14671,
 11463,
 15909,
 2271,
 2828,
 2462,
 1010,
 23760,
 9905,
 4244,
 27833,
 17577,
 101

In [ ]:
tokenizer.decode(inputs["input_ids"][0])